In [1]:
import os
import yaml
import pandas as pd
import pickle
import matplotlib.pyplot as plt

In [2]:
import pymrmr
import time

In [3]:
import sys
sys.path.append('../src')

In [4]:
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import GenericUnivariateSelect, chi2, f_classif, mutual_info_classif
from sklearn.preprocessing import MinMaxScaler

In [5]:
# Getting back the objects:
with open('../config/lists_radio.pkl', 'rb') as f:
    id_cols_radio,var_cols_radio,time_points,delta_time = pickle.load(f)

with open('../config/lists_path.pkl', 'rb') as f:
    id_cols_path,var_cols_path = pickle.load(f)

with open('../config/lists_imm.pkl', 'rb') as f:
    id_cols_imm,var_cols_imm = pickle.load(f)

with open('../config/lists_classEffect.pkl', 'rb') as f:
    var_cols_classEffect = pickle.load(f)
    
id_cols = id_cols_radio + id_cols_path + id_cols_imm + ['Class_name']
var_cols = var_cols_radio + var_cols_path + var_cols_imm
#var_cols = var_cols_imm + var_cols_path + var_cols_radio
    
#Retreive local data path
with open('../config/paths.yaml','r') as file:
    paths_list = yaml.safe_load(file)
    PATH = os.path.abspath(paths_list['PATH'])
    
# Open cleaned data
data = pd.read_csv(os.path.join(PATH,'tables','data_merge246.csv'))
data.head()

,Subject,StudyDay,Abolute_Neutrophil,Absolute_Basophil,Absolute_Eosinophil,Absolute_Lymphocyte,Absolute_Monocyte,Absolute_Reticulocyte,Age_d,Alanine_Aminotransferase,...,IL-8,MCP-1,MIP-1alpha,MIP-1beta,STUDY_NO_imm,TGF-alpha,TNF-alpha,VEGF,sCD40L,Class_name
0,B03757,2_delta,-2.90,-0.02,0.00,-1.28,0.39,-36.0,1602.0,-1.0,...,124.26,510.64,36.33,22.94,NaN,1.84,2.14,7.28,604.50,Infected
1,B03757,4_delta,-2.93,-0.02,0.03,-0.55,0.82,-38.2,1604.0,4.0,...,165.65,393.94,33.04,18.52,NaN,10.47,18.12,3.46,689.80,Infected
2,B03757,6_delta,-2.18,-0.01,0.04,-0.67,0.87,-52.8,1606.0,-8.0,...,90.74,242.25,53.29,10.73,NaN,78.16,421.31,280.70,528.85,Infected
3,B03781,2_delta,-6.32,-0.01,-0.06,-2.27,-0.37,-28.2,1600.0,25.0,...,67.59,103.44,2.00,2.20,NaN,0.00,15.23,-0.33,2.91,Infected
4,B03781,4_delta,-7.35,-0.01,-0.01,-1.61,-0.21,-29.4,1602.0,13.0,...,51.39,0.14,0.00,-1.41,NaN,0.00,1.17,-3.02,-9.84,Infected


In [6]:
X = data[var_cols]
y = data.Class

In [7]:
def show_list(l):
    i=0
    for t in l:
        i += 1
        print(str(i)+') '+t)

In [8]:
def get_modality(name):
    if name in var_cols_radio:
        return 'Radiomics'
    elif name in var_cols_path:
        return 'Clinical Pathology'
    elif name in var_cols_imm:
        return 'Immunology'

In [9]:
show_list(var_cols)

1) VoxelVolume-shape
2) Maximum3DDiameter-shape
3) Compactness2-shape
4) MeshVolume-shape
5) MajorAxisLength-shape
6) Sphericity-shape
7) LeastAxisLength-shape
8) Elongation-shape
9) Compactness1-shape
10) Flatness-shape
11) SurfaceArea-shape
12) MinorAxisLength-shape
13) Maximum2DDiameterColumn-shape
14) SphericalDisproportion-shape
15) Maximum2DDiameterRow-shape
16) JointAverage-glcm
17) Autocorrelation-glcm
18) JointEntropy-glcm
19) ClusterShade-glcm
20) Idmn-glcm
21) JointEnergy-glcm
22) Contrast-glcm
23) DifferenceVariance-glcm
24) Idn-glcm
25) Idm-glcm
26) Correlation-glcm
27) SumAverage-glcm
28) SumEntropy-glcm
29) MCC-glcm
30) SumSquares-glcm
31) ClusterProminence-glcm
32) Imc2-glcm
33) Imc1-glcm
34) DifferenceAverage-glcm
35) Id-glcm
36) ClusterTendency-glcm
37) GrayLevelVariance-gldm
38) HighGrayLevelEmphasis-gldm
39) DependenceEntropy-gldm
40) DependenceNonUniformity-gldm
41) GrayLevelNonUniformity-gldm
42) SmallDependenceHighGrayLevelEmphasis-gldm
43) LargeDependenceHighGra

### Feature Selection: test different scoring functions

(Lists are not ranked)

In [10]:
k=34

In [11]:
mask = GenericUnivariateSelect(score_func=f_classif,mode='k_best', param=k).fit(X, y).get_support()
var_cols_fbestk = X.columns[mask].to_list()
print('----- f-stat -----')
#show_list(var_cols_fbestk)

----- f-stat -----


In [12]:
mask = GenericUnivariateSelect(score_func=mutual_info_classif,mode='k_best', param=k).fit(X, y).get_support()
var_cols_MIbestk = X.columns[mask].to_list()
print('----- Mutual Information -----')
show_list(var_cols_MIbestk)

----- Mutual Information -----
1) JointAverage-glcm
2) Autocorrelation-glcm
3) JointEntropy-glcm
4) Idmn-glcm
5) SumAverage-glcm
6) SumEntropy-glcm
7) HighGrayLevelEmphasis-gldm
8) DependenceEntropy-gldm
9) GrayLevelNonUniformity-gldm
10) LargeDependenceHighGrayLevelEmphasis-gldm
11) LowGrayLevelEmphasis-gldm
12) InterquartileRange-firstorder
13) Skewness-firstorder
14) Uniformity-firstorder
15) Median-firstorder
16) RobustMeanAbsoluteDeviation-firstorder
17) RootMeanSquared-firstorder
18) Entropy-firstorder
19) Mean-firstorder
20) ShortRunLowGrayLevelEmphasis-glrlm
21) LowGrayLevelRunEmphasis-glrlm
22) GrayLevelNonUniformityNormalized-glrlm
23) GrayLevelNonUniformity-glrlm
24) LongRunHighGrayLevelEmphasis-glrlm
25) HighGrayLevelRunEmphasis-glrlm
26) ZoneVariance-glszm
27) SizeZoneNonUniformity-glszm
28) LargeAreaEmphasis-glszm
29) LargeAreaLowGrayLevelEmphasis-glszm
30) Coarseness-ngtdm
31) Complexity-ngtdm
32) Strength-ngtdm
33) Busyness-ngtdm
34) IL-1ra


In [13]:
X_scaled = MinMaxScaler().fit(X).transform(X) #scale to 0-1
mask = GenericUnivariateSelect(score_func=chi2,mode='k_best', param=k).fit(X_scaled, y).get_support()
var_cols_chibestk = X.columns[mask].to_list()
print('----- Chi-squared -----')
#show_list(var_cols_chibestk)

----- Chi-squared -----


In [14]:
var_cols_intersectbestk = list(set(var_cols_fbestk) & set(var_cols_MIbestk) & set(var_cols_chibestk))
print('----- f, MI, chi intersection -----')
#show_list(var_cols_intersectbestk)

----- f, MI, chi intersection -----


### Feature Selection: Percentile (10%)

In [15]:
# mask = GenericUnivariateSelect(score_func=f_classif,mode='fpr', param=0.05).fit(X, y).get_support()
# var_cols_MIbestk = X.columns[mask].to_list()
# print('----- False-positive rate -----')
# show_list(var_cols_MIbestk)

In [16]:
# mask = GenericUnivariateSelect(score_func=f_classif,mode='percentile', param=10).fit(X, y).get_support()
# var_cols_MIbestk = X.columns[mask].to_list()
# print('----- Mutual Information -----')
# show_list(var_cols_MIbestk)

### MRMR

In [17]:
#z-score normalize
data_znorm = data.copy()

for var in var_cols:
    data_znorm[var] = (data_znorm[var]-data_znorm[var].mean()) / data_znorm[var].std()

In [18]:
data_znorm.describe()

,Abolute_Neutrophil,Absolute_Basophil,Absolute_Eosinophil,Absolute_Lymphocyte,Absolute_Monocyte,Absolute_Reticulocyte,Age_d,Alanine_Aminotransferase,Albumin,AlbuminvGlobulin_Ratio,...,IL-6,IL-8,MCP-1,MIP-1alpha,MIP-1beta,STUDY_NO_imm,TGF-alpha,TNF-alpha,VEGF,sCD40L
count,6.000000e+01,6.000000e+01,6.000000e+01,6.000000e+01,6.000000e+01,6.000000e+01,60.000000,6.000000e+01,6.000000e+01,6.000000e+01,...,6.000000e+01,6.000000e+01,6.000000e+01,60.000000,6.000000e+01,0.0,59.000000,6.000000e+01,6.000000e+01,6.000000e+01
mean,7.123931e-17,-4.070818e-17,8.881784e-17,4.440892e-17,9.436896e-17,-1.480297e-17,1853.350000,3.885781e-17,-3.173387e-16,-2.164935e-16,...,-3.238150e-17,2.775558e-18,-1.017704e-16,0.000000,6.013708e-18,NaN,3.326102,2.347659e-17,-2.220446e-17,9.251859e-18
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,395.150211,1.000000e+00,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000,1.000000e+00,NaN,11.326384,1.000000e+00,1.000000e+00,1.000000e+00
min,-3.011986e+00,-2.664118e+00,-3.026466e+00,-2.388901e+00,-1.860477e+00,-2.365826e+00,1381.000000,-1.726354e+00,-3.161104e+00,-2.115343e+00,...,-1.151289e+00,-1.495320e+00,-9.614781e-01,-1.140231,-2.376417e+00,NaN,-1.660000,-8.383644e-01,-9.688209e-01,-5.929778e-01
25%,-3.712513e-01,-6.906973e-01,-5.534108e-01,-6.669514e-01,-7.670108e-01,-6.228530e-01,1525.500000,-8.917796e-01,-5.359657e-01,-5.316196e-01,...,-3.435243e-01,-5.380956e-01,-5.892673e-01,-0.400474,-3.947555e-01,NaN,-0.005000,-2.232490e-01,-2.988714e-01,-2.663390e-01
50%,-5.313024e-02,2.960131e-01,3.631759e-01,1.080439e-01,-1.299481e-01,1.200300e-01,1666.500000,3.683097e-02,1.203188e-01,2.493737e-01,...,-2.664754e-01,-4.140358e-01,-3.584290e-01,-0.362054,-6.738312e-02,NaN,0.000000,-2.232490e-01,-2.634173e-01,-2.507885e-01
75%,5.566934e-01,2.960131e-01,6.571754e-01,7.228578e-01,6.972825e-01,5.266758e-01,2193.250000,6.128046e-01,7.766033e-01,7.411593e-01,...,7.111817e-02,7.726685e-02,-2.942225e-03,0.001051,5.180285e-02,NaN,0.335000,-8.200671e-02,-6.201300e-02,-1.882937e-01
max,3.579393e+00,2.269434e+00,1.539174e+00,1.691013e+00,2.855689e+00,2.801306e+00,2506.000000,2.975472e+00,2.089172e+00,2.554389e+00,...,6.849505e+00,3.812268e+00,3.850666e+00,4.334546,4.154850e+00,NaN,78.160000,7.087168e+00,6.188417e+00,6.812384e+00


In [19]:
# var_cols = var_cols_radio + var_cols_path

# var_cols.remove('Blood_Urea_Nitrogen')
# var_cols.remove('Creatinine_')

In [20]:
#data_znorm.isna().any()

In [21]:
for var in var_cols:
    if data_znorm[var].isna().any():
        print(var)

### MRMR parameter definition

In [22]:
if False:
    # Choose either Mutual information quotient (MIQ) or Mutual information difference (MID)
    # Github is here: https://github.com/fbrundu/pymrmr

    k_tmp=35#len(data)/2
    #k_tmp=35
    print('k = ',k_tmp)
    #var_cols_mrmr = pymrmr.mRMR(data_znorm[['Class']+var_cols],'MIQ', k_tmp)
    var_cols_mrmr = pymrmr.mRMR(data_znorm[['Class']+var_cols],'MID', k_tmp)
    var_cols_mrmr
    print('Radiomics',[get_modality(n) for n in var_cols_mrmr].count('Radiomics'))
    print('Clinical Pathology',[get_modality(n) for n in var_cols_mrmr].count('Clinical Pathology'))
    print('Immunology',[get_modality(n) for n in var_cols_mrmr].count('Immunology'))
    
if False:
    tmp_df = pd.DataFrame({'var':var_cols_mrmr,'mod':[get_modality(n) for n in var_cols_mrmr]})
    tmp_df.to_csv(os.path.join(PATH,'tables','feature_selection_mrmr.csv'))
    print(os.path.join(PATH,'tables','feature_list_mrmr.csv'))

### MRMR Permute Call

In [23]:
recompute_mrmr_permute = False

if recompute_mrmr_permute:
    from ml_classic_repo import mrmr_permute
    corr_mrmr = mrmr_permute.mrmr_permute(data_znorm = data_znorm,var_cols = var_cols,a_thresh = 0.05)
    corr_mrmr.to_csv(os.path.join(PATH,'tables','mrmr_permute_05_20211020_demo.csv'))
else:
    corr_mrmr = pd.read_csv(os.path.join(PATH,'tables','mrmr_permute_05_20211013.csv'))

corr_mrmr['Modality'] = [get_modality(n) for n in corr_mrmr['Name'].tolist()]
    
display(corr_mrmr) #.Name.values.tolist()
#print('k='+str(len(corr_mrmr)))

,Unnamed: 0,Name,Score_thresh,Score,Modality
0,0,Uniformity-firstorder,-0.058,0.393,Radiomics
1,1,Blood_Urea_Nitrogen,0.011,0.078,Clinical Pathology
2,2,Busyness-ngtdm,-0.062,0.063,Radiomics
3,3,SphericalDisproportion-shape,-0.025,0.050,Radiomics
4,4,Absolute_Basophil,-0.001,0.042,Clinical Pathology
5,5,GrayLevelVariance-glszm,-0.028,0.048,Radiomics
6,6,ZoneVariance-glszm,-0.008,0.056,Radiomics
7,7,Contrast-ngtdm,-0.016,0.021,Radiomics
8,9,GrayLevelNonUniformityNormalized-glrlm,-0.119,0.052,Radiomics
9,10,Maximum3DDiameter-shape,-0.029,0.010,Radiomics


In [24]:
#k_mp = len(corr_mrmr)

In [25]:
#stop_right_here

In [26]:
#corr_mrmr.to_csv(os.path.join(PATH,'tables','mrmr_permute005b.csv'))

### This is where var_cols_FS is defined

In [27]:
# --- Define var_cols_FS ---
#var_cols_FS = var_cols_fbestk
#var_cols_FS = var_cols_intersectbestk
#var_cols_FS=['Idmn','GrayLevelNonUniformity','Uniformity','Entropy','GrayLevelNonUniformityNormalized','LongRunLowGrayLevelEmphasis','ZoneVariance','LargeAreaEmphasis','Busyness']
#var_cols_FS = var_cols_classEffect
#var_cols_FS = var_cols_mrmr
var_cols_FS = corr_mrmr.Name.values.tolist()

# --- Define status ---
#Write status message, change this anytime this part of the analysis changes
#status='FS:2x4ANOVA,Bonferroni'
#status='FS:2x3ANOVA,notCorrected'
#status='FS:2x3ANOVA,fdr'
#status='FS:ftest k=34'
#status='FS:MRMR MID k='+str(k_tmp)
#status='FS:MRMR MIQ k='+str(k_tmp)
status='MRMR-permute 0.05'

with open(os.path.join('..','config','analysis_status','x_feature_selection.txt'),'w') as out_file:
    out_file.write(status)

# ---- save lists ---
with open('../config/lists_FeatureSelect.pkl', 'wb') as f:
    pickle.dump(var_cols_FS, f)

#### Blood_Urea_Nitrogen and Creatinine_ not included in future studies

In [28]:
var_cols_FS

['Uniformity-firstorder',
 'Blood_Urea_Nitrogen',
 'Busyness-ngtdm',
 'SphericalDisproportion-shape',
 'Absolute_Basophil',
 'GrayLevelVariance-glszm',
 'ZoneVariance-glszm',
 'Contrast-ngtdm',
 'GrayLevelNonUniformityNormalized-glrlm',
 'Maximum3DDiameter-shape',
 'Median-firstorder',
 'Absolute_Reticulocyte',
 'LargeAreaEmphasis-glszm',
 'Entropy-firstorder',
 'MeshVolume-shape',
 'JointEnergy-glcm',
 'SumEntropy-glcm',
 'JointAverage-glcm',
 'Compactness1-shape',
 'Flatness-shape',
 'Skewness-firstorder',
 'VoxelVolume-shape',
 'SmallAreaHighGrayLevelEmphasis-glszm',
 'Sphericity-shape',
 'DependenceEntropy-gldm',
 'SumAverage-glcm',
 'InterquartileRange-firstorder',
 'GrayLevelNonUniformity-gldm',
 'Mean-firstorder',
 'SmallDependenceHighGrayLevelEmphasis-gldm',
 'GrayLevelNonUniformity-glszm',
 'RootMeanSquared-firstorder',
 'SizeZoneNonUniformity-glszm',
 'SurfaceArea-shape',
 'Maximum2DDiameterColumn-shape']

In [29]:
len(var_cols_FS)

35

In [30]:
anova_table = pd.read_csv(os.path.join(PATH,'tables','anova_join.csv'))
anova_table = corr_mrmr.set_index('Name').join(anova_table.set_index('Unnamed: 0'))
anova_table['Class_name_pval']
#anova_table.to_csv(os.path.join(PATH,'tables','anova_join_mrmr.csv'))

Name
Uniformity-firstorder                        0.000025
Blood_Urea_Nitrogen                          0.001450
Busyness-ngtdm                               0.000059
SphericalDisproportion-shape                 0.025633
Absolute_Basophil                            0.007634
GrayLevelVariance-glszm                      0.167575
ZoneVariance-glszm                           0.000104
Contrast-ngtdm                               0.002922
GrayLevelNonUniformityNormalized-glrlm       0.000222
Maximum3DDiameter-shape                      0.019405
Median-firstorder                            0.000884
Absolute_Reticulocyte                        0.027916
LargeAreaEmphasis-glszm                      0.000104
Entropy-firstorder                           0.000250
MeshVolume-shape                             0.000717
JointEnergy-glcm                             0.001305
SumEntropy-glcm                              0.000461
JointAverage-glcm                            0.001308
Compactness1-shape     

In [31]:
anova_table

,Unnamed: 0,Score_thresh,Score,Modality,Class_name_pval,StudyDay_pval,Interaction_pval,Class_name_np2,StudyDay_np2,Interaction_np2
Name,,,,,,,,,,
Uniformity-firstorder,0,-0.058,0.393,Radiomics,0.000025,0.008030,0.884410,0.637246,0.235117,0.006801
Blood_Urea_Nitrogen,1,0.011,0.078,Clinical Pathology,0.001450,0.026650,0.607202,0.439227,0.182405,0.027336
Busyness-ngtdm,2,-0.062,0.063,Radiomics,0.000059,0.161305,0.694058,0.601459,0.096391,0.020084
SphericalDisproportion-shape,3,-0.025,0.050,Radiomics,0.025633,0.068345,0.332384,0.247459,0.138488,0.059358
Absolute_Basophil,4,-0.001,0.042,Clinical Pathology,0.007634,0.045929,0.001975,0.333791,0.157303,0.292453
GrayLevelVariance-glszm,5,-0.028,0.048,Radiomics,0.167575,0.291107,0.273935,0.103051,0.066262,0.069410
ZoneVariance-glszm,6,-0.008,0.056,Radiomics,0.000104,0.147466,0.571721,0.576184,0.100883,0.030584
Contrast-ngtdm,7,-0.016,0.021,Radiomics,0.002922,0.546195,0.876293,0.396632,0.033041,0.007310
GrayLevelNonUniformityNormalized-glrlm,9,-0.119,0.052,Radiomics,0.000222,0.029307,0.746269,0.540419,0.178076,0.016128


### Exploring MRMR

#### MRMR on all measures, top 20, MIQ

In [32]:
pymrmr.mRMR(data_znorm[['Class']+var_cols],'MIQ', 20)

['Uniformity-firstorder',
 'Total_Protein',
 'Blood_Urea_Nitrogen',
 'TNF-alpha',
 'Contrast-ngtdm',
 'ZoneVariance-glszm',
 'GrayLevelVariance-glszm',
 'Busyness-ngtdm',
 'Absolute_Basophil',
 'SphericalDisproportion-shape',
 'GrayLevelNonUniformityNormalized-glrlm',
 'MeshVolume-shape',
 'Absolute_Reticulocyte',
 'Median-firstorder',
 'LargeAreaLowGrayLevelEmphasis-glszm',
 'Creatinine_',
 'Entropy-firstorder',
 'White_Blood_Cell_',
 'JointEnergy-glcm',
 'LargeAreaEmphasis-glszm']

#### MRMR only on radiomics measures

In [33]:
pymrmr.mRMR(data_znorm[['Class']+var_cols_radio],'MIQ', 20)

['Uniformity-firstorder',
 'Contrast-ngtdm',
 '10Percentile-firstorder',
 'ZoneVariance-glszm',
 'Maximum3DDiameter-shape',
 'GrayLevelVariance-glszm',
 'Sphericity-shape',
 'SumEntropy-glcm',
 'Busyness-ngtdm',
 'LargeAreaLowGrayLevelEmphasis-glszm',
 'MinorAxisLength-shape',
 'GrayLevelNonUniformityNormalized-glrlm',
 'Median-firstorder',
 'GrayLevelNonUniformityNormalized-glszm',
 'LargeAreaEmphasis-glszm',
 'Entropy-firstorder',
 'Flatness-shape',
 'VoxelVolume-shape',
 'SphericalDisproportion-shape',
 'JointEnergy-glcm']

In [34]:
pymrmr.mRMR(data_znorm[['Class']+var_cols_radio],'MIQ', 20)

['Uniformity-firstorder',
 'Contrast-ngtdm',
 '10Percentile-firstorder',
 'ZoneVariance-glszm',
 'Maximum3DDiameter-shape',
 'GrayLevelVariance-glszm',
 'Sphericity-shape',
 'SumEntropy-glcm',
 'Busyness-ngtdm',
 'LargeAreaLowGrayLevelEmphasis-glszm',
 'MinorAxisLength-shape',
 'GrayLevelNonUniformityNormalized-glrlm',
 'Median-firstorder',
 'GrayLevelNonUniformityNormalized-glszm',
 'LargeAreaEmphasis-glszm',
 'Entropy-firstorder',
 'Flatness-shape',
 'VoxelVolume-shape',
 'SphericalDisproportion-shape',
 'JointEnergy-glcm']

#### MRMR only on clinical pathology measures

In [35]:
pymrmr.mRMR(data_znorm[['Class']+var_cols_path],'MIQ', 20)

['Absolute_Basophil',
 'Absolute_Eosinophil',
 'Creatinine_',
 'Blood_Urea_Nitrogen',
 'White_Blood_Cell_',
 'Absolute_Reticulocyte',
 'Platelet',
 'Hematocrit',
 'Mean_Corpuscular_Hemoglobin_Concentraion',
 'Abolute_Neutrophil',
 'Aspartate_Aminotransferase',
 'Total_Protein',
 'Plateletcrit',
 'Mean_Corpuscular_Hemoglobin',
 'Absolute_Monocyte',
 'AlbuminvGlobulin_Ratio',
 'Hemoglobin',
 'Lactate_Dehydrogenase',
 'Red_Cell_Distrubtion_Width_-_Coefficient',
 'Albumin']

#### MRMR only on immunology measures

In [36]:
pymrmr.mRMR(data_znorm[['Class']+var_cols_imm],'MIQ', 20)

['MIP-1beta',
 'IL-17a',
 'TNF-alpha',
 'IFN-Alpha',
 'IL-8',
 'MCP-1',
 'IL-1b',
 'VEGF',
 'IL-1ra',
 'GM-CSF',
 'IL-6',
 'sCD40L',
 'IL-18',
 'IL-5',
 'IL-15',
 'IL-2',
 'MIP-1alpha',
 'IL-12_23_(p40)',
 'IFN-gamma',
 'IL-10']

#### MRMR on all measures MID

In [37]:
pymrmr.mRMR(data_znorm[['Class']+var_cols],'MID', 20)

['Uniformity-firstorder',
 'Blood_Urea_Nitrogen',
 'Busyness-ngtdm',
 'SphericalDisproportion-shape',
 'Absolute_Basophil',
 'GrayLevelVariance-glszm',
 'ZoneVariance-glszm',
 'Contrast-ngtdm',
 'TNF-alpha',
 'GrayLevelNonUniformityNormalized-glrlm',
 'Maximum3DDiameter-shape',
 'Median-firstorder',
 'White_Blood_Cell_',
 'Absolute_Reticulocyte',
 'LargeAreaEmphasis-glszm',
 'Entropy-firstorder',
 'Creatinine_',
 'IL-6',
 'MeshVolume-shape',
 'JointEnergy-glcm']

#### Display MRMR results next to variable type

In [38]:
feature_type=pd.DataFrame()

feature_type['Feature']=pymrmr.mRMR(data_znorm[['Class']+var_cols],'MID', 10)
feature_type['Rank'] = [i + 1 for i in feature_type.index.tolist()]

feature_type.loc[feature_type['Feature'].isin(var_cols_radio),'Type']='Radiomics'
feature_type.loc[feature_type['Feature'].isin(var_cols_path),'Type']='Clinical Pathology'
feature_type.loc[feature_type['Feature'].isin(var_cols_imm),'Type']='Immunology'

feature_type[['Rank','Type','Feature']]

,Rank,Type,Feature
0,1,Radiomics,Uniformity-firstorder
1,2,Clinical Pathology,Blood_Urea_Nitrogen
2,3,Radiomics,Busyness-ngtdm
3,4,Radiomics,SphericalDisproportion-shape
4,5,Clinical Pathology,Absolute_Basophil
5,6,Radiomics,GrayLevelVariance-glszm
6,7,Radiomics,ZoneVariance-glszm
7,8,Radiomics,Contrast-ngtdm
8,9,Immunology,TNF-alpha
9,10,Radiomics,GrayLevelNonUniformityNormalized-glrlm


In [39]:
# with open('var_cols_path_list.txt', 'w') as f:
#     for item in var_cols_path:
#         f.write("%s\n" % item)

In [40]:
data_znorm[['Class']+var_cols].set_index('Class').to_csv(os.path.join(PATH,'tables','data_znorm.csv'))

In [41]:
pymrmr.mRMR(data_znorm[['Class']+var_cols],'MID', 10)

['Uniformity-firstorder',
 'Blood_Urea_Nitrogen',
 'Busyness-ngtdm',
 'SphericalDisproportion-shape',
 'Absolute_Basophil',
 'GrayLevelVariance-glszm',
 'ZoneVariance-glszm',
 'Contrast-ngtdm',
 'TNF-alpha',
 'GrayLevelNonUniformityNormalized-glrlm']

In [42]:
# import mrmr_wrapper

# max_rel_df,mRMR_df = mrmr_wrapper.mrmr(data_znorm[['Class']+var_cols].set_index('Class'),'MID', 25)

In [43]:
# mRMR_df = mRMR_df.set_index('Name')
# max_rel_df = max_rel_df.set_index('Name')
# mRMR_df['max_rel'] = max_rel_df.Score
# mRMR_df['redundency'] = mRMR_df.max_rel - mRMR_df.Score
# mRMR_df

In [44]:
# data_znorm[['Class']+var_cols]